# L shaped hip roof builder

In this workshop we take as model an roof to define a function that takes in input a list of *verts* and a list of *cells* that are going to define the roof shape and produces in output an *HPC object* that represents the roof with his frame.

## The model

As model for this workshop I chose an *L shaped hip roof*, which can be seen at [this link](http://4.bp.blogspot.com/-BiJMpyiXYfQ/U4CMXsTEmiI/AAAAAAAABzs/i-LN0Apl0ok/s1600/Hip+and+Valley.jpg) or in the following images: ![first model](l_shaped_roof.png)![second model](second_l_shaped_roof.png)

## The method

The first step is to produce a good and coherent input for the function, because a bad serie of vertices and cells would lead to a *"non-roof-shaped" model*. In particular, all the vertices of a single cell must be **complanar** to have a real building, and for this the function ***check_complanarity(verts, cells)*** that takes the 2 lists in input and return *True* if all the cells are complanar and so we can use the inputs to build a proper roof: 

def check_complanarity(verts, cells):
	for cell in cells:
		if(len(cell) > 3):
			matrix = []
			lastPoint = cell[-1]
			for elem in cell:
				point = verts[int(elem)-1]
				row = []
				for i in range(len(point)):
					row.append(point[i]-verts[lastPoint-1][i])
				matrix.append(row)
			A = numpy.matrix(matrix)
			dim = numpy.linalg.matrix_rank(A)
			if(dim > 2):
				return False
	return True

There is also a support function ***normalize_list(verts)*** useful to round the vertices if they are derived from a *SKEL_2* of an HPC model for it would change a little the coordinates and we wouldn't have complanar cells:

In [2]:
def normalize_list(verts):
	for j in range(len(verts)):
		for i in range(len(verts[j])):
			if(abs(verts[j][i]) < 0.001):
				verts[j][i] = 0
			else:
				verts[j][i] = round(verts[j][i],1)

There is an another support function ***compute_incident_faces(listUkpol)*** that takes in input the list calculated from the call of *UKPOL* and returns a dictionary that has *keyvalue* as the coordinates of a vertex and as *value* a list of the cells incident in that point:

In [3]:
def compute_incident_faces(listUkpol):
	dictionary = {}
	verts = listUkpol[0]
	normalize_list(verts)
	cells = listUkpol[1]
	for cell in cells:
		for elem in cell:
			point = str(verts[int(elem)-1])
			if(point not in dictionary):
				dictionary[point] = []
			dictionary[point].append(elem)
	return dictionary